# M5 Forecast: Poisson Loss

This kernel serves as a Python translation of my [data.table](https://www.kaggle.com/mayer79/m5-forecast-attack-of-the-data-table) script, including the [dark magic](https://www.kaggle.com/kyakovlev/m5-dark-magic) calibration step. The results are not identical though.

Feature construction and lightGBM parameters were influenced by the two excellent kernels [Very fst Model](https://www.kaggle.com/ragnar123/very-fst-model) and [M5 ForecasteR](https://www.kaggle.com/kailex/m5-forecaster-0-57330).

One of the major performance boosts of the [data.table](https://www.kaggle.com/mayer79/m5-forecast-attack-of-the-data-table) script came from switching to Poisson loss. As far as I know, that kernel was the first public one to use this loss. Poisson loss is a typical way to model counts (e.g. number of sold items) and amounts to optimize the same objective function as a Poisson regression, i.e. the log likelihood derived from the Poisson distribution, or, (up to a constant) the Poisson deviance. The latter is defined as
$$
\sum_{observations} \left(y \log\frac{y}{predicted} - (y - predicted)\right),
$$
where the second part cancels out for an unbiased model.

Interestingly and in contrast to using MSE loss, it seems to work very well together with above mentioned [dark magic](https://www.kaggle.com/kyakovlev/m5-dark-magic) calibration step.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import os
from tqdm.notebook import tqdm

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

## Load data

In [ ]:
path = "inputs/"

calendar = pd.read_csv(os.path.join(path, "calendar.csv"))
selling_prices = pd.read_csv(os.path.join(path, "sell_prices.csv"))
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sales = pd.read_csv(os.path.join(path, "sales_train_validation.csv"))

## Describe and prepare data

We will now go through all data sets and prepare them for modelling.

### Calendar data

For each date (covering both training and test data), we have access to useful calendar information.

In [ ]:
calendar.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def prep_calendar(df):
    df = df.drop(["weekday", "event_name_2", "event_type_2"], axis=1)
    df = df.assign(d = df.d.str[2:].astype(int))
    to_ordinal = ["event_name_1", "event_type_1"] 
    df[to_ordinal] = df[to_ordinal].fillna("1")
    df[to_ordinal] = OrdinalEncoder(dtype="int").fit_transform(df[to_ordinal]) + 1
    to_int8 = ["wday", "month", "snap_CA", "snap_TX", "snap_WI"] + to_ordinal
    df[to_int8] = df[to_int8].astype("int8")
    return df

calendar = prep_calendar(calendar)

### Selling prices

Contains selling prices for each store_id, item_id_wm_yr_wk combination.

In [ ]:
selling_prices.head()

Derive some time related features:

In [ ]:
def prep_selling_prices(df):
    gr = df.groupby(["store_id", "item_id"])["sell_price"]
    df["sell_price_rel_diff"] = gr.pct_change()
    df["sell_price_cumrel"] = (gr.shift(0) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    df["sell_price_roll_sd7"] = gr.transform(lambda x: x.rolling(7).std())
    to_float32 = ["sell_price", "sell_price_rel_diff", "sell_price_cumrel", "sell_price_roll_sd7"]
    df[to_float32] = df[to_float32].astype("float32")
         
    return df

selling_prices = prep_selling_prices(selling_prices)

### Sales data

Contains the number of sold items (= our response) as well as some categorical features.

In [ ]:
sales.head()

#### Reshaping

We now reshape the data from wide to long, using "id" as fixed and swapping "d_x" columns. Along this process, we also add structure for submission data and reduce data size.

In [ ]:
def reshape_sales(df, drop_d = None):
    if drop_d is not None:
        df = df.drop(["d_" + str(i+1) for i in range(drop_d-1)], axis=1)
    df = df.assign(id=df.id.str.replace("_validation", ""))
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(1913 + i + 1) for i in range(2 * 28)])
    df = df.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                 var_name='d', value_name='demand')
    df = df.assign(d=df.d.str[2:].astype("int64"))

    return df

sales = reshape_sales(sales, 1000)

In [ ]:
sales.head()

#### Add time-lagged features

Add some of the derived features from kernel https://www.kaggle.com/ragnar123/very-fst-model.

In [ ]:
def prep_sales(df):
    df['lag_t28'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    df['lag_t29'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    df['lag_t30'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    df['lag_t31'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(31))

    df['rolling_mean_t7'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    df['rolling_mean_t30'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    df['rolling_mean_t60'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(60).mean())
    df['rolling_mean_t90'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    df['rolling_mean_t180'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    df['rolling_std_t7'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    df['rolling_std_t30'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
  
    to_float32 = ['lag_t28', 'rolling_mean_t7', 'rolling_mean_t30', 'rolling_mean_t60', 
                  'rolling_mean_t90', 'rolling_mean_t180', 'rolling_std_t7', 'rolling_std_t30']
    df[to_float32] = df[to_float32].astype("float32")
    
    # Remove rows with NAs except for submission rows. rolling_mean_t180 was selected as it produces most missings
    df = df[(df.d >= 1914) | (pd.notna(df.rolling_mean_t180))]
 
    return df

sales = prep_sales(sales)

### Combine data sources

In [ ]:
sales = sales.merge(calendar, how="left", on="d")
gc.collect()

In [ ]:
sales = sales.merge(selling_prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])
sales.drop(["wm_yr_wk"], axis=1, inplace=True)
gc.collect()
del selling_prices

In [ ]:
sales['week'] = sales['date'].astype('datetime64').dt.week

In [ ]:
sales['weekofmonth'] = np.ceil(sales['wday'] // 7).astype('int8')

In [ ]:
sales['day'] = sales['date'].astype('datetime64').dt.day

In [ ]:
sales

In [ ]:
c1 = sales['week'] == 13
c2 = sales['week'] == 14
c3 = sales['week'] == 15
c4 = sales['week'] == 16

c5 = sales['week'] == 17
c6 = sales['week'] == 18
c7 = sales['week'] == 19
c8 = sales['week'] == 20

sales = sales[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8]

## Prepare data for LightGBM interface

### Ordinal encoding of remaining categoricals

In [ ]:
for i, v in tqdm(enumerate(["item_id", "dept_id", "store_id", "cat_id", "state_id"])):
    sales[v] = OrdinalEncoder(dtype="int").fit_transform(sales[[v]]).astype("int16") + 1
gc.collect()

In [ ]:
sales.columns

In [ ]:
# Covariables used
x = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
       'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31', 'rolling_mean_t7',
       'rolling_mean_t30', 'rolling_mean_t60', 'rolling_mean_t90',
       'rolling_mean_t180', 'rolling_std_t7', 'rolling_std_t30',
       'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA',
       'snap_TX', 'snap_WI', 'sell_price', 'sell_price_rel_diff',
       'sell_price_cumrel', 'sell_price_roll_sd7', 'week', 'weekofmonth',
       'day']

In [ ]:
len(x)

#### Separate submission data and reconstruct id columns

In [ ]:
test = sales[sales.d >= 1914]
test = test.assign(id=test.id + "_" + np.where(test.d <= 1941, "validation", "evaluation"),
                   F="F" + (test.d - 1913 - 28 * (test.d > 1941)).astype("str"))

# Reduce sales
sales1 = sales[sales.d < 1914]
gc.collect()

#### Make training data

In [ ]:
import lightgbm as lgb

In [ ]:
# One month of validation data
flag = sales1.d >= 1914 - 28
valid = lgb.Dataset(sales1[flag][x], label = sales1[["demand"]][flag])
gc.collect()

# Rest is used for training
sales1 = sales1[~flag].drop(["d", "id"], axis=1)
del flag
gc.collect()
sales1 = lgb.Dataset(sales1[x], label = sales1[["demand"]])

# Trick to avoid memory spike when LightGBM converts everything to float32:
#   See https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
sales1.save_binary('train.bin')
sales1 = lgb.Dataset('train.bin')

## The model

In [ ]:
sa

In [ ]:
params = {
    'metric': 'rmse',
    'objective': 'poisson',
    'seed': 20,
    'learning_rate': 0.08,
    'lambda': 0.1,
    'num_leaves': 63,
    'bagging_fraction': 0.7,
    'bagging_freq': 1, 
    'colsample_bytree': 0.7
}

fit = lgb.train(params, 
                sales1, 
                num_boost_round = 2100, 
                valid_sets = [valid], 
                early_stopping_rounds = 400,
                verbose_eval = 100)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(sales[x], sales[["demand"]])

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = sales[x].iloc[train_index], sales[x].iloc[valid_index]
    y_train, y_valid = sales[["demand"]].iloc[train_index], sales[["demand"]].iloc[valid_index]
    
    sales = lgb.Dataset(X_train, label = y_train)
    sales.save_binary('train.bin')
    sales = lgb.Dataset('train.bin')
    
    fit = lgb.train(params, 
                sales, 
                num_boost_round = 2100, 
                valid_sets = [valid], 
                early_stopping_rounds = 400,
                verbose_eval = 100)

    pred = fit.predict(test[x])
    
    # 예측
    y_preds += pred / n_fold
    
    
    



In [ ]:
lgb.plot_importance(fit, importance_type="gain", precision=0, height=0.5, figsize=(6, 10));

## Submission

In [ ]:
test["demand"] = y_preds
submission = test.pivot(index="id", columns="F", values="demand").reset_index()[sample_submission.columns]

for i in range(1,29):
    submission['F'+str(i)] *= 1.0315
    submission.to_csv('submissions/submission.csv', index = False)

submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)